In [49]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/NLP Hotel Reviews'
df = pd.read_csv("movie_metadata.csv")
#https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata
#https://www.kaggle.com/datasets/carolzhangdc/imdb-5000-movie-dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/NLP Hotel Reviews


In [50]:
#tadf = df.head(10)
df

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660


In [51]:
df.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [52]:
features = ['actor_1_name','actor_2_name','genres','director_name']

In [53]:
def combine_features(row):
    return row['actor_1_name']+" "+row['actor_2_name']+" "+row['genres']+" "+row['director_name']
    print(combine_features)

In [54]:
df['actor_1_name'].unique()

array(['CCH Pounder', 'Johnny Depp', 'Christoph Waltz', ...,
       'Natalie Zea', 'Eva Boehnke', 'John August'], dtype=object)

In [55]:
df.dtypes

color                         object
director_name                 object
num_critic_for_reviews       float64
duration                     float64
director_facebook_likes      float64
actor_3_facebook_likes       float64
actor_2_name                  object
actor_1_facebook_likes       float64
gross                        float64
genres                        object
actor_1_name                  object
movie_title                   object
num_voted_users                int64
cast_total_facebook_likes      int64
actor_3_name                  object
facenumber_in_poster         float64
plot_keywords                 object
movie_imdb_link               object
num_user_for_reviews         float64
language                      object
country                       object
content_rating                object
budget                       float64
title_year                   float64
actor_2_facebook_likes       float64
imdb_score                   float64
aspect_ratio                 float64
m

In [56]:
#Combine features into single string so that they can be 
#represented by a single line on a graph (for cosine similarity)
# Using apply() method to combine two columns of text
def combine_features(row):
    return row['actor_1_name']+" "+row['actor_2_name']+" "+row['genres']+" "+row['director_name']
#df["combined_features"] = df[['actor_1_name','actor_2_name','genres','director_name']].apply(" ".join, axis=1)

In [57]:
df

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660


In [58]:
#Clean the data so there are no NaN values (Preprocessing)
for feature in features:
    df[feature] = df[feature].fillna('')

In [74]:
#Apply the combined_features method to the dataset
df["combined_features"] = df.apply(combine_features,axis=1)
print("Combined Features:", df["combined_features"].head())

Combined Features: 0    CCH Pounder Joel David Moore Action|Adventure|...
1    Johnny Depp Orlando Bloom Action|Adventure|Fan...
2    Christoph Waltz Rory Kinnear Action|Adventure|...
3    Tom Hardy Christian Bale Action|Thriller Chris...
4       Doug Walker Rob Walker Documentary Doug Walker
Name: combined_features, dtype: object


In [60]:
#Using the strings, created by combined_features, create a count matrix (for the creation of the similarity matrix) 
cv = CountVectorizer() 
count_matrix = cv.fit_transform(df["combined_features"])

#Using the count matrix, create a similarity matrix  using cosine simialrity
cosine_sim = cosine_similarity(count_matrix)

In [72]:
print(count_matrix)

  (0, 33)	1
  (0, 55)	1
  (0, 993)	1
  (0, 1110)	1
  (0, 1552)	1
  (0, 2072)	1
  (0, 2138)	1
  (0, 3044)	1
  (0, 3153)	1
  (0, 4442)	1
  (0, 5049)	1
  (0, 5617)	1
  (1, 33)	1
  (1, 55)	1
  (1, 723)	1
  (1, 1636)	1
  (1, 2072)	1
  (1, 2501)	1
  (1, 3165)	1
  (1, 4781)	1
  (1, 6423)	1
  (2, 33)	1
  (2, 55)	1
  (2, 1222)	1
  (2, 3426)	1
  :	:
  (5039, 6797)	1
  (5040, 587)	1
  (5040, 744)	1
  (5040, 1798)	1
  (5040, 2029)	1
  (5040, 2872)	1
  (5040, 4125)	1
  (5040, 4439)	1
  (5040, 5326)	1
  (5040, 6199)	1
  (5041, 101)	1
  (5041, 1324)	1
  (5041, 1511)	2
  (5041, 1798)	1
  (5041, 2766)	1
  (5041, 2890)	1
  (5041, 5363)	1
  (5041, 5421)	1
  (5042, 381)	1
  (5042, 858)	1
  (5042, 1731)	1
  (5042, 2589)	1
  (5042, 2787)	1
  (5042, 3163)	1
  (5042, 3175)	1


In [71]:
cosine_sim

array([[1.        , 0.28867513, 0.19245009, ..., 0.        , 0.        ,
        0.        ],
       [0.28867513, 1.        , 0.22222222, ..., 0.        , 0.        ,
        0.        ],
       [0.19245009, 0.22222222, 1.        , ..., 0.11111111, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.11111111, ..., 1.        , 0.10050378,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.10050378, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [61]:
df['movie_title']

0                                                 Avatar 
1               Pirates of the Caribbean: At World's End 
2                                                Spectre 
3                                  The Dark Knight Rises 
4       Star Wars: Episode VII - The Force Awakens    ...
                              ...                        
5038                             Signed Sealed Delivered 
5039                           The Following             
5040                                A Plague So Pleasant 
5041                                    Shanghai Calling 
5042                                   My Date with Drew 
Name: movie_title, Length: 5043, dtype: object

In [62]:
# remove special character
df.columns = df.columns.str.replace(' ', '')
df['movie_title'] = df['movie_title'].str.replace('\xa0', '', regex=True) #fills all NaNs with a blank string and replaces \xa0 (out of bounds error will happen if not replaced)

In [63]:
#These functions simply help get the title from the index and the index from the title  
def get_title_from_index(index):
  return df.loc[index, "movie_title"]

def get_index_from_title(title):
  try:
    return df.loc[df.movie_title == title].index[0]
  except IndexError:
    return print ("\nPlease select a different Movie\nHere are the Top Rated movies:\n")

In [64]:
#Asks for user imput to return movies that the user likes
moviename = input("\nType in a movie title: ")
#Takes the user input so the program can locate the row title is on
movie_index = get_index_from_title(moviename)


Type in a movie title: Tangled


In [65]:
#Produces the similarity scores that the program can search to find most similar movies
similar_movies = list(enumerate(cosine_sim[movie_index]))

In [69]:
similar_movies

[(0, 0.16012815380508716),
 (1, 0.1849000654084097),
 (2, 0.09245003270420485),
 (3, 0.0),
 (4, 0.0),
 (5, 0.08770580193070293),
 (6, 0.19611613513818402),
 (7, 0.9999999999999998),
 (8, 0.08362420100070908),
 (9, 0.2631174057921088),
 (10, 0.08770580193070293),
 (11, 0.08770580193070293),
 (12, 0.09805806756909201),
 (13, 0.1849000654084097),
 (14, 0.09245003270420485),
 (15, 0.16724840200141816),
 (16, 0.2631174057921088),
 (17, 0.08362420100070908),
 (18, 0.1849000654084097),
 (19, 0.3076923076923077),
 (20, 0.19611613513818402),
 (21, 0.1849000654084097),
 (22, 0.08770580193070293),
 (23, 0.19611613513818402),
 (24, 0.2773500981126146),
 (25, 0.17541160386140586),
 (26, 0.09805806756909201),
 (27, 0.08362420100070908),
 (28, 0.08362420100070908),
 (29, 0.08006407690254358),
 (30, 0.09245003270420485),
 (31, 0.2773500981126146),
 (32, 0.08362420100070908),
 (33, 0.2773500981126146),
 (34, 0.16012815380508716),
 (35, 0.41812100500354543),
 (36, 0.08770580193070293),
 (37, 0.087705801

In [66]:
#Sorts the similarity scores so the program can return the best scores first
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

In [68]:
sorted_similar_movies

[(228, 0.6689936080056728),
 (792, 0.5384615384615385),
 (1747, 0.5384615384615385),
 (3177, 0.5017452060042545),
 (125, 0.4803844614152615),
 (403, 0.4803844614152615),
 (582, 0.4803844614152615),
 (971, 0.4803844614152615),
 (1913, 0.4803844614152615),
 (4224, 0.4803844614152615),
 (902, 0.44474958999666075),
 (1284, 0.44474958999666075),
 (55, 0.43852900965351466),
 (119, 0.43852900965351466),
 (175, 0.43852900965351466),
 (276, 0.43852900965351466),
 (1912, 0.43852900965351466),
 (2332, 0.43852900965351466),
 (2241, 0.4193139346887673),
 (35, 0.41812100500354543),
 (43, 0.41812100500354543),
 (56, 0.41812100500354543),
 (87, 0.41812100500354543),
 (107, 0.41812100500354543),
 (209, 0.41812100500354543),
 (238, 0.41812100500354543),
 (262, 0.41812100500354543),
 (266, 0.41812100500354543),
 (277, 0.41812100500354543),
 (298, 0.41812100500354543),
 (334, 0.41812100500354543),
 (384, 0.41812100500354543),
 (419, 0.41812100500354543),
 (423, 0.41812100500354543),
 (509, 0.4181210050035

In [67]:
#Returns the most similar movies based on the user's input
i=0
print("\nSimilar movies to "+moviename+" are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i=i+1
    if i>9:
        break


Similar movies to Tangled are:

Stuart Little 2
Sinbad: Legend of the Seven Seas
Aladdin
Teacher's Pet
Frozen
Enchanted
Shrek 2
The Book of Life
Rugrats Go Wild
The Brave Little Toaster
